In [3]:
!pwd

/Users/shenqingyun/Desktop/git_repository/DMAC/dmac/playground


In [29]:
import json
import numpy as np
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer

# Load the data
data = []
with open('../../data/exp1_data/train_data.txt', 'r') as file:
    for line in file:
        item = json.loads(line)
        data.append(item)

print("Load complete")
print(data[0])

Load complete
{'label': 0, 'raw': "I only watched the Wanda Sykes portion of this show. I think it was interesting to watch because it was before she came out as a lesbian. She was married to a man at the time. She actually made some jokes that raised my eyebrows since she is now a lesbian. I didn't like this because it seemed hypocritical but I think Wanda Sykes is really funny. She is one of the few comedians who can make me really LOL (Laugh Out Loud). If you want to see what her comedy was like before she came out a lesbian or you are a Wanda Sykes fan, watch it. I am a huge fan of hers. I would like to see her in a live how. I am glad she is true to herself now and came out as a lesbian. I hope Wanda keeps on making me and others laugh for a long time to come."}


In [65]:
# Split data into labels and raw text
labels = [item["label"] for item in data]
texts = [item["raw"] for item in data]

# Word Embedding using Gensim Word2Vec
# You may need to preprocess and tokenize the text before training the Word2Vec model.
# Here, we'll assume simple whitespace-based tokenization.
tokenized_texts = [text.split() for text in texts]
print(len(tokenized_texts[0]))
embedding_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, sg=0)

156


In [60]:
# Function to convert text to a vector representation using Word Embeddings
def text_to_vector(text, model):
    words = text.split()
    # Aggregate word vectors (skip words not in the vocabulary)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# train_data = [data[0], data[1]]
# res = np.array([text_to_vector(item["raw"], embedding_model) for item in train_data])
# print(res)

In [64]:
# Create a binary label representation using MultiLabelBinarizer
# mlb = MultiLabelBinarizer()
# binary_labels = mlb.fit_transform([[label] for label in labels])

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_idx, dev_idx in kf.split(data):
    # Split data into training and development sets
    train_data = [data[i] for i in train_idx]
    dev_data = [data[i] for i in dev_idx]

    train_label = [labels[idx] for idx in train_idx]

    dev_label = [labels[idx] for idx in dev_idx]

    # Convert text data to word vectors
    train_vectors = np.array([text_to_vector(item["raw"], embedding_model) for item in train_data])
    # print(train_vectors[0])
    dev_vectors = np.array([text_to_vector(item["raw"], embedding_model) for item in dev_data])

    assert len(train_label) == len(train_vectors)
    assert len(dev_label) == len(dev_vectors)

    # Train a multilabel SVM classifier with Gaussian kernel
    clf = SVC(kernel='rbf', C=1.0, gamma="scale")
    clf.fit(train_vectors, train_label)
    # clf.fit(train_vectors, binary_labels[train_idx])

    # Make predictions on the development set
    dev_predictions = clf.predict(dev_vectors)
    print(dev_predictions)

    # Calculate accuracy and store it
    accuracy = accuracy_score(dev_label, dev_predictions)
    accuracies.append(accuracy)

# Calculate and print the mean accuracy across folds
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy:.2f}")


[3 4 0 ... 9 9 9]
[4 1 9 ... 9 6 9]
[1 9 1 ... 9 9 9]
[4 6 3 ... 0 9 9]
[5 1 1 ... 7 9 0]
Mean Accuracy: 0.37


Yep that works. Lets tune the hyperparams and test polynomial and RBF kernels in the main code